SQL y PySpark se usan para transformar y analizar datos, pero difieren en cómo y dónde se ejecutan:

SQL es un lenguaje declarativo que se ejecuta directamente sobre bases de datos. Es ideal para consultas estructuradas, filtrados y agregaciones simples.

PySpark usa Python para manipular datos distribuidos en clústeres. Permite trabajar con volúmenes masivos de datos y escribir transformaciones complejas con una sintaxis similar a SQL, pero ejecutadas en paralelo y de forma escalable.

👉 En resumen: SQL es más simple y directo, mientras que PySpark está diseñado para Big Data y procesamiento distribuido.

In [0]:
 from pyspark.sql.functions import *

In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.people (
  id INT,
  name STRING,
  age INT,
  city STRING
)
USING DELTA;


In [0]:
%sql
INSERT INTO people (id, name, age, city) VALUES
(1, 'Ana', 23, 'Bogotá'),
(2, 'Luis', 35, 'Medellín'),
(3, 'Sofía', 29, 'Bogotá'),
(4, 'Diego', 17, 'Cali');

num_affected_rows,num_inserted_rows
4,4


In [0]:
people_sdf = spark.table("people")

Seleccionar columnas

In [0]:
%sql
SELECT name, city FROM people;

name,city
Ana,Bogotá
Luis,Medellín
Sofía,Bogotá
Diego,Cali


In [0]:
people_sdf.select("name", "city").show()

+-----+--------+
| name|    city|
+-----+--------+
|  Ana|  Bogotá|
| Luis|Medellín|
|Sofía|  Bogotá|
|Diego|    Cali|
+-----+--------+



Filtro condicional

In [0]:
%sql
SELECT * FROM people WHERE age >= 18;

id,name,age,city
1,Ana,23,Bogotá
2,Luis,35,Medellín
3,Sofía,29,Bogotá


In [0]:
people_sdf.filter(people_sdf.age >= 18).show()

+---+-----+---+--------+
| id| name|age|    city|
+---+-----+---+--------+
|  1|  Ana| 23|  Bogotá|
|  2| Luis| 35|Medellín|
|  3|Sofía| 29|  Bogotá|
+---+-----+---+--------+



Ordenar resultados

In [0]:
%sql
SELECT * FROM people ORDER BY age DESC;

id,name,age,city
2,Luis,35,Medellín
3,Sofía,29,Bogotá
1,Ana,23,Bogotá
4,Diego,17,Cali


In [0]:
people_sdf.orderBy(people_sdf.age.desc()).show()

+---+-----+---+--------+
| id| name|age|    city|
+---+-----+---+--------+
|  2| Luis| 35|Medellín|
|  3|Sofía| 29|  Bogotá|
|  1|  Ana| 23|  Bogotá|
|  4|Diego| 17|    Cali|
+---+-----+---+--------+



Crear columna calculada

In [0]:
%sql
SELECT *, age * 12 AS age_months FROM people;

id,name,age,city,age_months
1,Ana,23,Bogotá,276
2,Luis,35,Medellín,420
3,Sofía,29,Bogotá,348
4,Diego,17,Cali,204


In [0]:
people_sdf.withColumn("age_months", people_sdf.age * 12).show()

+---+-----+---+--------+----------+
| id| name|age|    city|age_months|
+---+-----+---+--------+----------+
|  1|  Ana| 23|  Bogotá|       276|
|  2| Luis| 35|Medellín|       420|
|  3|Sofía| 29|  Bogotá|       348|
|  4|Diego| 17|    Cali|       204|
+---+-----+---+--------+----------+



Agregación por grupo

In [0]:
%sql
SELECT city, AVG(age) AS avg_age FROM people GROUP BY city;

city,avg_age
Bogotá,26.0
Medellín,35.0
Cali,17.0


In [0]:
people_sdf.groupBy("city") \
    .agg(avg(col("age")).alias("avg_age")) \
    .show()

+--------+-------+
|    city|avg_age|
+--------+-------+
|  Bogotá|   26.0|
|Medellín|   35.0|
|    Cali|   17.0|
+--------+-------+



Contar cuántas personas hay por ciudad

In [0]:
%sql
SELECT city, COUNT(*) AS total_people FROM people GROUP BY city;


city,total_people
Bogotá,2
Medellín,1
Cali,1


In [0]:
people_sdf.groupBy("city").agg(count("*").alias("total_people")).show()

+--------+------------+
|    city|total_people|
+--------+------------+
|  Bogotá|           2|
|Medellín|           1|
|    Cali|           1|
+--------+------------+



Filtrar y agrupar al mismo tiempo

In [0]:
%sql
SELECT city, AVG(age) AS avg_age
FROM people
WHERE age >= 18
GROUP BY city;


city,avg_age
Bogotá,26.0
Medellín,35.0


In [0]:
people_sdf.filter(people_sdf.age >= 18) \
    .groupBy("city") \
    .agg(avg("age").alias("avg_age")) \
    .show()


+--------+-------+
|    city|avg_age|
+--------+-------+
|  Bogotá|   26.0|
|Medellín|   35.0|
+--------+-------+



Concatenar nombre y ciudad


In [0]:
%sql
SELECT CONCAT(name, ' - ', city) AS name_city FROM people;


name_city
Ana - Bogotá
Luis - Medellín
Sofía - Bogotá
Diego - Cali


In [0]:
people_sdf.select(concat("name", lit(" - "), "city").alias("name_city")).show()

+---------------+
|      name_city|
+---------------+
|   Ana - Bogotá|
|Luis - Medellín|
| Sofía - Bogotá|
|   Diego - Cali|
+---------------+



Contar personas mayores o menores de edad

In [0]:
%sql
SELECT 
  SUM(CASE WHEN age >= 18 THEN 1 ELSE 0 END) AS adults,
  SUM(CASE WHEN age < 18 THEN 1 ELSE 0 END) AS minors
FROM people;


adults,minors
3,1


In [0]:
people_sdf.select(
    sum(when(people_sdf.age >= 18, 1).otherwise(0)).alias("adults"),
    sum(when(people_sdf.age < 18, 1).otherwise(0)).alias("minors")
).show()

+------+------+
|adults|minors|
+------+------+
|     3|     1|
+------+------+



**🚀 ¡Nos vemos en @caro.data!**

Hasta aquí llegamos con este notebook, pero hay mucho más por aprender.

Sigamos explorando juntos el mundo de los datos.

Nos vemos en el próximo notebook 👩‍💻💫